In [1]:
import numpy as np 
import tensorflow as tf

#load data set
x_real = np.loadtxt("train_x.csv", delimiter=",") # load from text 
y_real = np.loadtxt("train_y.csv", delimiter=",") 
x_test_real = np.loadtxt("test_x.csv", delimiter=",") # load from text 

#x = x.reshape(-1, 64, 64) # reshape 
#y = y.reshape(-1, 1) 

In [59]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
# lb = preprocessing.LabelBinarizer()
# lb.fit(classes)


# x_real_1 = np.where(x_real > 252, 1, 0)
x_real_1 = x_real/255
x_test_real = x_test_real/255
#normalize value in x_train
x_train, x_test, y_train, y_test = train_test_split(x_real_1, y_real, test_size=0.2, random_state=42)
lb = preprocessing.LabelBinarizer()
lb.fit(classes)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)
print(y_train.shape,y_test.shape)

(40000, 40) (10000, 40)


In [ ]:
#keras
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D , AveragePooling2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization

# input image dimensions
img_rows, img_cols = 64, 64

batch_size = 100
num_classes = 40
epochs = 1

# input image dimensions
img_rows, img_cols = 64, 64

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#start building layers

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(512, (3, 3), padding="same"))
# model.add(Activation('relu'))

# model.add(Conv2D(512, (3, 3), padding="same"))
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))


model.add(Dropout(0.25))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.02),
              metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

from keras.callbacks import ModelCheckpoint

filepath="checkpoint.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list, verbose=1,validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

# save_model(model,"model3.json","model3.h5")
# load_model = load_model("model3.json","model3.h5")
# output_result(load_model,"kaggle_output_model3.csv")

x_train shape: (40000, 64, 64, 1)
40000 train samples
10000 test samples
Train on 40000 samples, validate on 10000 samples
Epoch 1/1
 1200/40000 [..............................] - ETA: 67s - loss: 3.6871 - acc: 0.0242

In [67]:
# save_model(model,"model2.json","model2.h5")
# load_model = load_model("model.json","model.h5")
# output_result(load_model,"kaggle_output_model1_2.csv")

Loaded model from disk


In [63]:
def save_model(model_to_save, json_name,h5_name):
#     serialize model to JSON
    model_json = model_to_save.to_json()
    with open(json_name, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_to_save.save_weights(h5_name)
    print("Saved model to disk")

    
from keras.models import model_from_json  

def load_model(json_name,h5_name):
    # load json and create model
    json_file = open(json_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(h5_name)
    print("Loaded model from disk")
    return loaded_model
 
def output_result(loaded_model,file_name):
    global x_test_real
    
    x_test_real = x_test_real.reshape(x_test_real.shape[0], img_rows, img_cols,1)
    output_kaggle = loaded_model.predict(x_test_real)
    output_kaggle = lb.inverse_transform(output_kaggle)

    with open(file_name,"w") as output:
        output.write("Id")
        output.write(",")
        output.write("Label")
        output.write("\n")
        for row in range(len(output_kaggle)):
            output.write(str(row+1))
            output.write(",")
            output.write(str(output_kaggle[row]))
            output.write("\n")
# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(x_test, y_test, verbose=1)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [7]:
# #keras
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten,Activation
# from keras.layers import Conv2D, MaxPooling2D , AveragePooling2D
# from keras import backend as K
# from keras.layers.normalization import BatchNormalization

# batch_size = 100
# num_classes = 40
# epochs = 30

# # input image dimensions
# img_rows, img_cols = 64, 64

# x_train, x_test, y_train, y_test = train_test_split(x_real_1, y_real, test_size=0.25, random_state=42)
# lb = preprocessing.LabelBinarizer()
# lb.fit(classes)
# y_train = lb.transform(y_train)
# y_test = lb.transform(y_test)


# def build_model(conv,conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout):
#     global x_train, x_test, y_train, y_test
#     if K.image_data_format() == 'channels_first':
#         x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#         x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#         input_shape = (1, img_rows, img_cols)
#     else:
#         x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#         x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#         input_shape = (img_rows, img_cols, 1)

#     x_train = x_train.astype('float32')
#     x_test = x_test.astype('float32')
#     x_train /= 255
#     x_test /= 255
#     print('x_train shape:', x_train.shape)
#     print(x_train.shape[0], 'train samples')
#     print(x_test.shape[0], 'test samples')

#     # convert class vectors to binary class matrices
#     # y_train = keras.utils.to_categorical(y_train, num_classes)
#     # y_test = keras.utils.to_categorical(y_test, num_classes)

#     model = Sequential()

#     model.add(Conv2D(conv[0][0], kernel_size=(conv[0][1], conv[0][1]),padding='same',input_shape=input_shape))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
    
#     for conv_filter, kernel_size in conv[1:]:
#         if conv_filter == None:
#             if is_pool_max:
#                 model.add(MaxPooling2D(pool_size=(2, 2)))
#             else:
#                 model.add(AveragePooling2D(pool_size=(2, 2)))
#         else:
#             model.add(Conv2D(conv_filter, (kernel_size, kernel_size),padding='same'))
#             if batchNorm:
#                 model.add(BatchNormalization())
#             model.add(Activation('relu'))
    

#     for conv_filter, kernel_size in conv2:
#         if conv_filter == None:
#             if is_pool_max:
#                 model.add(MaxPooling2D(pool_size=(2, 2)))
#             else:
#                 model.add(AveragePooling2D(pool_size=(2, 2)))
#         else:
#             model.add(Conv2D(conv_filter, (kernel_size, kernel_size),padding='same'))
#             if batchNorm:
#                 model.add(BatchNormalization())
#             model.add(Activation('relu'))

# #     model.add(Dropout(0.5))
#     model.add(Flatten())

#     for num_neuron in dense:
        
#         model.add(Dense(num_neuron))
#         if batchNorm:
#                 model.add(BatchNormalization())
#         model.add(Activation('relu'))
#         if dropout:
#             model.add(Dropout(0.5))
   

#     model.add(Dense(num_classes, activation='softmax'))

#     from keras import optimizers
#     if optimizer == "sdg":
#         optimizer = optimizers.SGD(lr=learning_rate)
#     elif optimizer == "adam":
#         optimizer = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#     else:
#         optimizer = optimizers.Adadelta(lr=learning_rate, rho=0.95, epsilon=1e-08, decay=0.0)
#     # model.compile(loss='mean_squared_error', optimizer=sgd)
#     model.compile(loss=keras.losses.categorical_crossentropy,
#                   optimizer=optimizer,
#                   metrics=['accuracy'])

#     model.fit(x_train, y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               verbose=1,
#               validation_data=(x_test, y_test))

#     score = model.evaluate(x_test, y_test, verbose=0)
    
#     print('Test loss:', score[0])
#     print('Test accuracy:', score[1])
#     return (model, score)


# convs = [
#     [(32,5),(32,3),(32,1),(None, None)],
#     [(32,5),(32,5),(32,5),(None, None)],
#     [(64,5),(32,3),(None, None)]
# ]
# models = []
# for conv in convs:
#     for conv2 in convs:
#         for dense in [[256],[512,256],[512,512,256]]:
#             for batchNorm in (True,):
#                 for learning_rate in [0.1]:
#                     for is_pool_max in (True,False):
#                         for optimizer in ["sgd"]:
#                             for dropout in (True,):
#                                 print(len(models) ,conv, conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout)
#                                 models.append(build_model(conv,conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout))
#                                 print(models[-1][0])



0 [(32, 5), (32, 3), (32, 1), (None, None)] [(32, 5), (32, 3), (32, 1), (None, None)] [256] True 0.1 False sgd True
x_train shape: (37500, 64, 64, 1)
37500 train samples
12500 test samples
Train on 37500 samples, validate on 12500 samples
Epoch 1/30
37500/37500 [==============================] - 58s - loss: 4.1265 - acc: 0.0491 - val_loss: 3.7108 - val_acc: 0.0383
Epoch 2/30
37500/37500 [==============================] - 53s - loss: 3.7541 - acc: 0.0870 - val_loss: 3.4872 - val_acc: 0.1149
Epoch 3/30
37500/37500 [==============================] - 53s - loss: 3.5905 - acc: 0.1050 - val_loss: 3.4483 - val_acc: 0.1206
Epoch 4/30
37500/37500 [==============================] - 53s - loss: 3.4706 - acc: 0.1203 - val_loss: 3.4118 - val_acc: 0.1196
Epoch 5/30
37500/37500 [==============================] - 54s - loss: 3.3667 - acc: 0.1338 - val_loss: 3.4035 - val_acc: 0.1242
Epoch 6/30
25700/37500 [===================>..........] - ETA: 15s - loss: 3.2663 - acc: 0.1503

KeyboardInterrupt: 

In [ ]:
# #keras
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten,Activation
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# batch_size = 128
# num_classes = 10
# epochs = 12

# # input image dimensions
# img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),input_shape=input_shape))
# model.add(Activation('relu'))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Dropout(0.25))

# model.add(Flatten())

# model.add(Dense(128, activation='relu'))

# model.add(Dropout(0.5))

# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# score = model.evaluate(x_test, y_test, verbose=0)

# print('Test loss:', score[0])
# print('Test accuracy:', score[1])